In [1]:
import sys
import getpass
import re
from functools import partial
from tqdm import tqdm
import uuid

user = getpass.getuser()
sys.dont_write_bytecode = True

sys.path.insert(0, '/Users/{}/Box/DataViz Projects/Utility Code'.format(user))

from utils_io import *  # initial imports
reload_module('utils_io')  # reload after update to utils_io.py
from utils_io import *  # reload reflects updates

#geo imports
import geopandas as gp
import folium
from geopandas.tools import geocode
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from geopy import Point

## Match un-geocoded projects by APN

-[ParcelAtlas v24 (BASIS)](https://data.bayareametro.gov/Cadastral/ParcelAtlas-v24-BASIS-/8hp3-7hq6)

In [2]:
parcels_id = '8hp3-7hq6'

In [3]:
#Pull Sonoma County Parcels from Socrata
query_args = {'where': "county = 'Sonoma'"}
parcels_sonoma = pull_df_from_socrata(parcels_id,query_args=query_args)

pulling data in 9 chunks of 22000 rows each
pulling chunk 0
pulling chunk 1
pulling chunk 2
pulling chunk 3
pulling chunk 4
pulling chunk 5
pulling chunk 6
pulling chunk 7
pulling chunk 8
took 52.8932 seconds


In [4]:
#Pull un-geocoded sonoma projects from Socrata
apr_id = 'briv-ikjp'
query_args = {'where': "cnty_name = 'SONOMA'"}

In [5]:
sonoma_projects = pull_df_from_socrata(apr_id,query_args=query_args)

pulling data in 1 chunks of 3787 rows each
pulling chunk 0
took 3.4189 seconds


In [6]:
sonoma_projects[['mtc_id','apn','mtc_address_full','street_address','mtc_wkt']].sample(n=20)

,mtc_id,apn,mtc_address_full,street_address,mtc_wkt
85,d2b0ba0a-4b17-41cd-bacd-3792a5aa2819,002-152-008,"402PIPER ST., HEALDSBURG, CA , USA",402PIPER ST.,"{'type': 'Point', 'coordinates': [-122.8656688..."
2435,56a4c4fc-b048-4aab-a449-477a60491765,TBD,"TBD, HEALDSBURG, CA , USA",TBD,"{'type': 'Point', 'coordinates': [-122.8691609..."
1443,1efb75f5-547e-4d74-8a48-cd4f61b177b9,029-240-007,"3655 ROLLING OAKS RD, SANTA ROSA, SONOMA COUNT...","3655 ROLLING OAKS RD, SANTA ROSA CA 95404","{'type': 'Point', 'coordinates': [-122.676313,..."
2748,648bc151-355a-449f-a06e-7641bfd97f5d,159-590-071,"5315 KELLIANN PLACE, ROHNERT PARK, CA , USA",5315 KELLIANN PLACE,"{'type': 'Point', 'coordinates': [-122.676886,..."
2954,74f1fd25-6f82-462f-9015-76ab2b1fc594,157020009,NaN,NaN,NaN
3400,261e0404-4ef1-4683-b3e9-b2eda48be639,020-050-017,"1709 SPRING HILL ROAD, SONOMA COUNTY, CA , USA",1709 SPRING HILL ROAD,"{'type': 'Point', 'coordinates': [-122.6875376..."
2579,79d42bcf-4d67-4eaf-8a31-dcf374c0e4af,159-560-023,"5137 KOLTON PLACE, ROHNERT PARK, CA , USA",5137 KOLTON PLACE,"{'type': 'Point', 'coordinates': [-122.68008, ..."
1315,8f7552c0-d25d-42e0-a4d2-e0b435bfb7ef,127-211-009,"840 WEST NAPA STREET UNITS E & F BLDG. 4, SONO...",840 WEST NAPA STREET UNITS E & F BLDG. 4,"{'type': 'Point', 'coordinates': [-122.4739458..."
1073,3f9d9fa5-304d-4cd9-8bcf-351d5ebe7b1d,125031022,"1619 SENDERO LN, SANTA ROSA, CA , USA",1619 SENDERO LN,"{'type': 'Point', 'coordinates': [-122.7395491..."
3390,34b2dcab-ceec-4650-b5e2-5436f709bcf6,128-131-051,"164 NEWCOMB STREET, SONOMA, CA , USA",164 NEWCOMB STREET,"{'type': 'Point', 'coordinates': [-122.4619364..."


## Format APNs to match Sonoma County Pattern

In [8]:
#Subset apns that match the county pattern
regex_ctny_pattern = r'\d{3}-\d{3}-\d{3}'
sonoma_apn_reg = sonoma_projects[['apn']][sonoma_projects['apn'].str.contains(regex_ctny_pattern,na=False)]

In [9]:
#Create a column with formatted apn, setting it to the first correctly mattched pattern 
#if multiple apns listed in column
regex_ctny_extract_pattern = r'(^\d{3}-\d{3}-\d{3})'
sonoma_apn_reg['apn_fmt'] = sonoma_apn_reg['apn'].str.extract(regex_ctny_extract_pattern)

In [13]:
sonoma_apn_reg.shape

(2389, 2)

In [11]:
#update the main dataframe with updated values
sonoma_projects['mtc_apn_fmt'].update(sonoma_apn_reg['apn_fmt'])

In [14]:
sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['mtc_apn_fmt'].notnull()].sample(n=15)

,mtc_id,apn,mtc_apn_fmt
530,51e9a071-9737-440a-9cd5-8477354afcad,159-620-037,159-620-037
2370,0e1f4e52-9e3b-44e1-9a1f-f0cbbc409ed4,002-091-015,002-091-015
2134,1a95783d-f9f3-4b4c-b050-face346c98c5,082-143-078,082-143-078
2752,327d05a7-a4b9-497f-8576-0e7c7829716c,159-590-084,159-590-084
1853,b24acec8-8c89-464a-add8-c3692b4dea9f,058-300-035,058-300-035
2392,9cb7679c-7fad-41ea-a951-b38dc9e39a09,091-330-001,091-330-001
2617,df698a2a-ef1b-47f2-9a80-0f9c4e3201bb,159-570-010,159-570-010
1706,5d1ff2c1-0ac7-4586-b651-a4745ef41abe,055-193-006,055-193-006
1734,e0fbc78a-1a2e-41a4-9cb3-228db7a7116d,058-031-012,058-031-012
2857,dd3d73f5-176b-4628-b1b0-0750fd11b56f,159-610-084,159-610-084


In [15]:
#Select apns of length 8
#These apns need to be pre-pended with a leading 0 and seperated by dashes every 3 digits
sonoma_apn_eight_dig = sonoma_projects[['apn']].loc[sonoma_projects['apn'].str.len() == 8]

In [16]:
sonoma_apn_eight_dig.shape

(547, 1)

In [17]:
sonoma_apn_eight_dig['apn_fmt'] = '0' + sonoma_apn_eight_dig['apn']

In [18]:
sonoma_apn_eight_dig['apn_fmt'] = (sonoma_apn_eight_dig['apn_fmt'].str[0:3] + 
                                 '-' +
                                 sonoma_apn_eight_dig['apn_fmt'].str[3:6] +
                                 '-' +
                                 sonoma_apn_eight_dig['apn_fmt'].str[6:])

In [19]:
#Update original dataframe apn_fmt column with correctly formatted apn
sonoma_projects['mtc_apn_fmt'].update(sonoma_apn_eight_dig['apn_fmt'])

In [20]:
sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['apn'].str.len() == 8].sample(n=15)

,mtc_id,apn,mtc_apn_fmt
3176,be9fac4d-701d-4242-8850-19d95cff7ab2,34030013,034-030-013
765,6f3a53c3-5506-4ae4-b54a-20a0206fae88,10311028,010-311-028
807,e8a250d6-fb0f-4e80-9c0d-da657d9f943f,31180017,031-180-017
1166,73babc64-74a0-4505-a4cc-0bee96571458,15533046,015-533-046
3319,c14639fd-9439-4e8c-9e09-0bacbf2a12cb,38192025,038-192-025
812,209ac3dc-5ca5-4aef-989b-9b42531b6529,34030013,034-030-013
3179,e3c89d47-6527-4dd8-9d00-d05d396f55c4,34030013,034-030-013
963,45d34354-2b4a-43e7-8555-99e7800f80f7,59010009,059-010-009
801,34258d9c-214d-4dd2-92da-75f9f819792f,14670015,014-670-015
1234,e9e2e479-5d73-401c-8faf-5da53981abdb,32010044,032-010-044


In [44]:
#Extract 9 digit apns
regex_nine = r'^\d{9}'
sonoma_apn_nine_dig = sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['apn']
                                                                      .str.contains(regex_nine,na=False)]

In [45]:
sonoma_apn_nine_dig.shape

(639, 3)

In [48]:
#Extract 9 digit apns into apn fmt column but only the first occurance if multiple apns
regex_nine_extract = r'(^\d{9})'
sonoma_apn_nine_dig['apn_fmt'] = sonoma_apn_nine_dig['apn'].str.extract(regex_nine_extract)

In [49]:
sonoma_apn_nine_dig['apn_fmt'] = (sonoma_apn_nine_dig['apn_fmt'].str[0:3] + 
                                 '-' +
                                 sonoma_apn_nine_dig['apn_fmt'].str[3:6] +
                                 '-' +
                                 sonoma_apn_nine_dig['apn_fmt'].str[6:])

In [50]:
sonoma_apn_nine_dig

,mtc_id,apn,mtc_apn_fmt,apn_fmt
0,b3a34e42-6e61-4455-be49-7cf127087133,153590023,153-590-023,153-590-023
1,dfa4745a-2419-4663-a092-99fbefcba732,153590024,153-590-024,153-590-024
2,bc0537c3-54ea-4b2e-86db-be232c6498c7,153590025,153-590-025,153-590-025
3,370a4f7e-9e66-4881-bbd0-a4f944f2fcc2,153590026,153-590-026,153-590-026
4,b78008b8-b43a-475e-b445-de8df4b981c5,153600001,153-600-001,153-600-001
...,...,...,...,...
3163,224e13cc-91e9-44f5-be0b-e8a8891c30bf,182520079,182-520-079,182-520-079
3164,69c0f2d5-9f10-4623-85a0-87adcfa792b3,182520079,182-520-079,182-520-079
3165,867af865-b051-495a-b94e-dd54e8e528f1,183400046,183-400-046,183-400-046
3166,89728b5c-ac37-444f-9b02-3210b2c3df1c,183440021,183-440-021,183-440-021


In [51]:
#Update original dataframe apn_fmt column with correctly formatted apn
sonoma_projects['mtc_apn_fmt'].update(sonoma_apn_nine_dig['apn_fmt'])

In [52]:
sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['apn'].str.len() == 9].sample(n=15)

,mtc_id,apn,mtc_apn_fmt
1147,32846645-d8de-4018-b98c-ddeaf09381a7,125252004,125-252-004
3092,c1cd3513-8055-4bc6-8c97-70fc46795afe,157020009,157-020-009
2991,dbc17b4e-7755-4dfe-878b-eaa32ab44e30,125071014,125-071-014
1131,fd20ed78-99ac-41e0-a340-a174e01563d7,125071014,125-071-014
1134,f8145bd4-51a8-43aa-b299-32e927738df7,125071014,125-071-014
1126,1ab61e57-8cef-4306-9a90-3248a0656fe9,125071014,125-071-014
17,963da6d9-2b3b-4632-8a3e-0977093ea327,153610003,153-610-003
1058,2371ba93-fefd-4a93-aaee-8ec53d5b4d7b,125031022,125-031-022
307,dfae5aea-4229-4e5e-bf46-609be3befd77,137520014,137-520-014
1016,7cce66b4-4adc-4beb-a44c-de8cd3db69ad,125031022,125-031-022


In [53]:
sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['mtc_apn_fmt'].isnull()]

,mtc_id,apn,mtc_apn_fmt
2164,c05bf7a7-8973-4aba-812b-b76f3e3e22d3,10/1/87 0:00,NaN
2165,df15d76f-245d-46c2-8a76-4d95cdc515bf,10/13/33 0:00,NaN
2186,2d7de426-21c7-49da-9653-c35d58fdc705,11/15/67 0:00,NaN
2187,9a9175d7-757b-4a1f-8be0-c992e7d39bf3,11/2/67 0:00,NaN
2188,fb710eba-10d7-4160-abe7-412c4a09f825,11/20/67 0:00,NaN
...,...,...,...
2557,89307c5b-830c-4b66-b18d-aae408e8179a,NaN,NaN
2558,deb86930-6997-42fe-a742-c4627261a60a,NaN,NaN
2559,6942f9e6-84b2-4c4c-8c0d-55761ee1ec0e,NaN,NaN
2560,1403daa6-c2ab-4449-a7f0-c4efcc30fae4,NaN,NaN


In [54]:
#Subset projects with 7 digits 
sonoma_apn_seven_dig = sonoma_projects[['apn']][sonoma_projects['apn'].str.len() == 7]

In [55]:
#Prepend with two zeros
sonoma_apn_seven_dig['apn_fmt'] = '00' + sonoma_apn_seven_dig['apn']

In [56]:
sonoma_apn_seven_dig

,apn,apn_fmt
112,5192025,005192025
114,6071009,006071009
115,6241024,006241024
116,6450010,006450010
117,6562019,006562019
...,...,...
3346,9131009,009131009
3347,9272003,009272003
3348,9293001,009293001
3349,9344005,009344005


In [57]:
sonoma_apn_seven_dig['apn_fmt'] = (sonoma_apn_seven_dig['apn_fmt'].str[0:3] + 
                                 '-' +
                                 sonoma_apn_seven_dig['apn_fmt'].str[3:6] +
                                 '-' +
                                 sonoma_apn_seven_dig['apn_fmt'].str[6:])

In [58]:
sonoma_apn_seven_dig

,apn,apn_fmt
112,5192025,005-192-025
114,6071009,006-071-009
115,6241024,006-241-024
116,6450010,006-450-010
117,6562019,006-562-019
...,...,...
3346,9131009,009-131-009
3347,9272003,009-272-003
3348,9293001,009-293-001
3349,9344005,009-344-005


In [59]:
#Update original dataframe with updated apns
sonoma_projects['mtc_apn_fmt'].update(sonoma_apn_seven_dig['apn_fmt'])

In [67]:
sonoma_projects[['apn','mtc_apn_fmt']][sonoma_projects['mtc_apn_fmt'].isnull()].shape

(104, 2)

In [69]:
#Upsert formatted apns to socrata
upsert_df_socrata(sonoma_projects[['mtc_id','mtc_apn_fmt']],apr_id)

upserting data to Socrata in 1 chunks of 3787 rows each
upserted chunk 0
data upserted to briv-ikjp
took 2.9269 seconds


## Join to parcels by apn

In [76]:
#Subset projects to required fields
geocode_types = ['APPROXIMATE','GEOMETRIC_CENTER','NOT_GEOCODED']
sonoma_projects_sub = (sonoma_projects[['mtc_id','apn','mtc_apn_fmt']][sonoma_projects['mtc_geocode_type']
                                                                       .isin(geocode_types)]
                       .copy())

In [77]:
sonoma_projects_parcels = pd.merge(sonoma_projects_sub,
                                  parcels_sonoma,
                                  left_on ='mtc_apn_fmt',
                                  right_on='apn',
                                  how='left',
                                  indicator=True)

In [78]:
sonoma_projects_parcels[sonoma_projects_parcels['_merge'] == 'both'].shape

(114, 50)

In [80]:
sonoma_projects_parcels.shape

(216, 50)

In [81]:
sonoma_projects_parcels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216 entries, 0 to 215
Data columns (total 50 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   mtc_id       216 non-null    object  
 1   apn_x        216 non-null    object  
 2   mtc_apn_fmt  184 non-null    object  
 3   objectid     114 non-null    float64 
 4   apn_y        114 non-null    object  
 5   apn2         114 non-null    object  
 6   state        114 non-null    object  
 7   county       114 non-null    object  
 8   fips         114 non-null    object  
 9   sit_hse_nu   40 non-null     object  
 10  sit_dir      0 non-null      object  
 11  sit_str_na   50 non-null     object  
 12  sit_str_sf   50 non-null     object  
 13  sit_full_s   50 non-null     object  
 14  sit_city     50 non-null     object  
 15  sit_state    50 non-null     object  
 16  sit_zip      49 non-null     object  
 17  sit_zip4     36 non-null     object  
 18  land_value   114 non-null    f

In [82]:
sonoma_projects_parcels_sub = sonoma_projects_parcels[['mtc_id',
                                                       'mtc_apn_fmt',
                                                       'xcoord',
                                                       'ycoord',
                                                       '_merge']][sonoma_projects_parcels['_merge'] == 'both'].copy()

In [83]:
sonoma_projects_parcels_sub

,mtc_id,mtc_apn_fmt,xcoord,ycoord,_merge
13,64d9a665-fb38-48a2-8172-071fe5e69534,144-450-002,-122.704,38.3202,both
23,1ad6da8a-6d32-4ea5-a57a-4efb30e501e6,089-300-007,-122.876,38.6229,both
26,ee196048-9612-4657-8892-1d34daa3a56f,159-610-054,-122.676,38.3245,both
27,d4208131-6f04-46be-ac8d-1b337a53496f,159-610-055,-122.676,38.3246,both
28,2d3f91d9-a65a-48fc-9e35-7d429b48e004,159-620-097,-122.674,38.3463,both
...,...,...,...,...,...
210,14fc32bb-ea34-482a-ab3d-52964b05a219,163-320-085,-122.792,38.5298,both
211,05764f97-0587-42b3-9dc4-30c44fdb3a79,163-320-086,-122.792,38.5299,both
212,bde40512-27e1-4167-8ee7-dbe09aca9b50,163-320-087,-122.792,38.5301,both
214,687092a9-3a24-4ad4-a8c1-4122c50a9e31,117-020-039,-122.997,38.7653,both


In [84]:
#Create geodataframe from lat long coords
sonoma_geo = gp.GeoDataFrame(sonoma_projects_parcels_sub,
                                  geometry=gp.points_from_xy(sonoma_projects_parcels_sub['xcoord'],
                                                             sonoma_projects_parcels_sub['ycoord']),
                                  crs="EPSG:4326")

In [85]:
sonoma_geo.head(5)

,mtc_id,mtc_apn_fmt,xcoord,ycoord,_merge,geometry
13,64d9a665-fb38-48a2-8172-071fe5e69534,144-450-002,-122.704,38.3202,both,POINT (-122.70400 38.32020)
23,1ad6da8a-6d32-4ea5-a57a-4efb30e501e6,089-300-007,-122.876,38.6229,both,POINT (-122.87600 38.62290)
26,ee196048-9612-4657-8892-1d34daa3a56f,159-610-054,-122.676,38.3245,both,POINT (-122.67600 38.32450)
27,d4208131-6f04-46be-ac8d-1b337a53496f,159-610-055,-122.676,38.3246,both,POINT (-122.67600 38.32460)
28,2d3f91d9-a65a-48fc-9e35-7d429b48e004,159-620-097,-122.674,38.3463,both,POINT (-122.67400 38.34630)


In [86]:
sonoma_geo['mtc_wkt'] = sonoma_geo['geometry'].map(lambda x: x.wkt)

In [87]:
#Rename columns to match final schema
sonoma_geo.rename(columns={'xcoord':'mtc_long','ycoord':'mtc_lat'},inplace=True)

In [88]:
#Add mtc_mapped column and set to True
sonoma_geo['mtc_mapped'] = True

In [89]:
#Add mtc_geocode_type and set to APN_MATCH
sonoma_geo['mtc_geocode_type'] = 'APN_MATCH'

In [90]:
sonoma_geo.head(5)

,mtc_id,mtc_apn_fmt,mtc_long,mtc_lat,_merge,geometry,mtc_wkt,mtc_mapped,mtc_geocode_type
13,64d9a665-fb38-48a2-8172-071fe5e69534,144-450-002,-122.704,38.3202,both,POINT (-122.70400 38.32020),POINT (-122.704 38.3202),True,APN_MATCH
23,1ad6da8a-6d32-4ea5-a57a-4efb30e501e6,089-300-007,-122.876,38.6229,both,POINT (-122.87600 38.62290),POINT (-122.876 38.6229),True,APN_MATCH
26,ee196048-9612-4657-8892-1d34daa3a56f,159-610-054,-122.676,38.3245,both,POINT (-122.67600 38.32450),POINT (-122.676 38.3245),True,APN_MATCH
27,d4208131-6f04-46be-ac8d-1b337a53496f,159-610-055,-122.676,38.3246,both,POINT (-122.67600 38.32460),POINT (-122.676 38.3246),True,APN_MATCH
28,2d3f91d9-a65a-48fc-9e35-7d429b48e004,159-620-097,-122.674,38.3463,both,POINT (-122.67400 38.34630),POINT (-122.674 38.3463),True,APN_MATCH


In [91]:
#Upsert updated records to socrata
final_columns = ['mtc_id','mtc_apn_fmt','mtc_long','mtc_lat','mtc_mapped','mtc_geocode_type','mtc_wkt']
upsert_df_socrata(sonoma_geo[final_columns],apr_id)

upserting data to Socrata in 1 chunks of 114 rows each
upserted chunk 0
data upserted to briv-ikjp
took 1.4472 seconds
